## Label Encoding

In [42]:
#importing the libraries
import pandas as pd
import numpy as np

In [43]:
df = pd.read_csv("Salary.csv")
# print the datatypes of features
print(df.info)

<bound method DataFrame.info of   Country  Age  Salary
0   India   44    7000
1      US   34    6500
2   Japan   45    8000
3      US   30    4300
4   Japan   32    2100>


In [44]:
# lable encoding
from sklearn import preprocessing
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'Country'. 
df['Country']= label_encoder.fit_transform(df['Country']) 
print(df)

   Country  Age  Salary
0        0   44    7000
1        2   34    6500
2        1   45    8000
3        2   30    4300
4        1   32    2100


## One-Hot Encoding

In [45]:
# importing one hot encoder 
from sklearn.preprocessing import OneHotEncoder
# creating one hot encoder object 
onehotencoder = OneHotEncoder()
#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 
X = onehotencoder.fit_transform(df.Country.values.reshape(-1,1)).toarray()
#To add this back into the original dataframe 
dfOneHot = pd.DataFrame(X, columns = ["Country_"+str(int(i)) for i in range(df.shape[1])]) 
df = pd.concat([df, dfOneHot], axis=1)
#droping the country column 
df= df.drop(['Country'], axis=1) 
#printing to verify 
print(df.head())


   Age  Salary  Country_0  Country_1  Country_2
0   44    7000        1.0        0.0        0.0
1   34    6500        0.0        0.0        1.0
2   45    8000        0.0        1.0        0.0
3   30    4300        0.0        0.0        1.0
4   32    2100        0.0        1.0        0.0


### Handle multicolinearity in ohe

In [46]:
# Function to calculate VIF
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
def calculate_vif(data):
    vif_df = pd.DataFrame(columns = ['Var', 'Vif'])
    x_var_names = data.columns
    for i in range(0, x_var_names.shape[0]):
        y = data[x_var_names[i]]
        x = data[x_var_names.drop([x_var_names[i]])]
        r_squared = sm.OLS(y,x).fit().rsquared
        vif = round(1/(1-r_squared),2)
        vif_df.loc[i] = [x_var_names[i], vif]
    return vif_df.sort_values(by = 'Vif', axis = 0, ascending=False, inplace=False)

X=df.drop(['Salary'],axis=1)
calculate_vif(X)



,Var,Vif
2,Country_1,33.05
3,Country_2,23.14
1,Country_0,21.93
0,Age,2.12


In [47]:
# drop one column
df = df.drop(df.columns[[0]], axis=1)
calculate_vif(df)

,Var,Vif
3,Country_2,3.94
2,Country_1,3.57
1,Country_0,3.47
0,Salary,1.13
